# Module 3: The Madden Oracle

## Stats vs Talent — We Need Validation

Okay, we have stats. But stats only tell us what *happened*, not necessarily how *talented* a player is.

A mediocre QB can have great stats in a great system, so we need an **external truth**.

We're using **Madden Ratings** as a proxy for raw talent.


In [ ]:
# Standard imports
import polars as pl
from pathlib import Path
import sys
import statsmodels

# Project setup
PROJECT_ROOT = Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT / "src"))

from ingestion import load_pbp_cached
from cleaning import normalize_player_name, standardize_team_abbr
from features import qb_season_stats
from viz import plot_madden_correlation, apply_dark_theme

import plotly.express as px

# Load data
pbp = load_pbp_cached(seasons=range(2000, 2025))
madden_raw = pl.read_csv(PROJECT_ROOT / "assets" / "madden_super_bowl.csv")

print("✅ Data loaded")
print(f"Madden records: {len(madden_raw)}")


## The Problem: Name Matching 😱

Our stats say: **"Patrick Mahomes"**

Madden says: **"P. Mahomes II"**

**Computers hate this.** Let's look at the raw data:


In [ ]:
# Preview Madden data - notice the name formats
print("Sample Madden Data:")
madden_raw.select(["player_name", "position", "team", "overall"]).head(10)


## 🖐️ HANDS ON: String Cleaning in Polars

In Pandas, you might use a lambda function or `.str.replace()`.

In Polars, we have the **`.str` namespace** — and it's beautiful.

Let's clean this string:


In [ ]:
# Clean player names using the .str namespace
madden_clean = madden_raw.with_columns(
    pl.col("player_name")
    .str.to_uppercase()           # "Patrick Mahomes II" -> "PATRICK MAHOMES II"
    .str.replace_all(r"[^A-Z ]", "")  # Remove non-letters (II, Jr., periods)
    .str.strip_chars()            # Trim whitespace
    .alias("name_clean")
)

print("Name Cleaning Results:")
madden_clean.select(["player_name", "name_clean"]).head(10)


### See How Clean That Chaining Is?

```python
pl.col("player_name")
.str.to_uppercase()
.str.replace_all(...)
.str.strip_chars()
```

**It reads like a sentence.** And it runs in parallel across all CPU cores!


In [ ]:
# More name cleaning examples
sample_names = pl.DataFrame({
    "original": [
        "Patrick Mahomes II",
        "Ja'Marr Chase", 
        "Travis Kelce ",
        "Tom Brady Jr.",
        "A.J. Brown"
    ]
})

# Apply the same cleaning
cleaned = sample_names.with_columns(
    pl.col("original")
    .str.to_uppercase()
    .str.replace_all(r"[^A-Z ]", "")
    .str.strip_chars()
    .alias("cleaned")
)

print("Name Normalization Examples:")
cleaned


## Joining Datasets

Now we join Madden with NFL data.

**Polars joins are ruthless efficiency.** No index management needed.


In [ ]:
# Calculate QB season stats from play-by-play data
qb_stats = (
    pbp
    .filter(
        pl.col("play_type") == "pass",
        pl.col("passer_id").is_not_null()
    )
    .group_by(["passer_id", "season"])
    .agg(qb_season_stats())
    .filter(pl.col("dropbacks") >= 200)  # Qualified QBs only
    .collect()
)

print(f"Qualified QB seasons: {len(qb_stats)}")
qb_stats.select(["player_name", "season", "team", "epa_per_dropback", "dropbacks"]).head(5)


In [ ]:
# Prepare Madden QBs for joining
madden_qbs = (
    madden_raw
    .filter(pl.col("position") == "QB")
    .with_columns([
        pl.col("season").cast(pl.Int32),
        normalize_player_name(pl.col("player_name")).alias("name_normalized")
    ])
)

# Add normalized names to NFL data too
qb_stats_normalized = qb_stats.with_columns(
    normalize_player_name(pl.col("player_name")).alias("name_normalized")
)

print(f"Madden QB records: {len(madden_qbs)}")


In [ ]:
# Join Madden + NFL data on normalized name and season
joined = (
    madden_qbs
    .join(
        qb_stats_normalized,
        on=["name_normalized", "season"],
        how="inner"
    )
)

print(f"✅ Successful matches: {len(joined)}")
print("\nJoined Data:")
joined.select([
    "player_name", "season", "super_bowl", "team",
    "overall", "epa_per_dropback", "dropbacks"
])


## Correlation: Does Madden Predict EPA?

The moment of truth — do the vibes match reality?


In [ ]:
# Calculate correlation between Madden OVR and EPA
correlation = joined.select(
    pl.corr("overall", "epa_per_dropback").alias("ovr_vs_epa"),
    pl.corr("overall", "total_epa").alias("ovr_vs_total_epa"),
)

print("📊 CORRELATION ANALYSIS")
print("=" * 50)
print(f"\n  Madden OVR vs EPA per Dropback: {correlation['ovr_vs_epa'][0]:.3f}")
print(f"  Madden OVR vs Total EPA:        {correlation['ovr_vs_total_epa'][0]:.3f}")
print("\n" + "=" * 50)


In [ ]:
# Visualize the relationship
fig = px.scatter(
    joined.to_pandas(),
    x="overall",
    y="epa_per_dropback",
    hover_name="player_name",
    size="dropbacks",
    title="Madden OVR vs Real NFL Performance (EPA)",
    trendline="ols"
)

apply_dark_theme(fig)
fig.update_layout(
    xaxis_title="Madden Overall Rating",
    yaxis_title="EPA per Dropback"
)
fig.show()


## 📊 Key Insight

There's a **positive correlation** between Madden ratings and actual performance. Now that we can see EA Sports ratings adjusters *do* know what they're doing, this gives us **external validation** we can use as a feature in our prediction model.


## Next Steps

We now have:
1. ✅ **QB Efficiency** — From play-by-play data (EPA)
2. ✅ **QB Talent** — From Madden ratings
3. ⏳ **Team Defense** — Coming next

Time to build our **Super Bowl Prediction Model**!

Continue to **[05_prediction_machine.ipynb](05_prediction_machine.ipynb)** →
